# Treinamento com interface de alto nível

## Importação das bibliotecas

In [ ]:
!pip install openpyxl scikit-learn

In [ ]:
# http://pytorch.org/
from os.path import exists

import torch

In [ ]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

# Importando bibliotecas para leitura do dataset
import pandas as pd

### Dataset escolhido
https://www.kaggle.com/datasets/muratkokludataset/date-fruit-datasets/data

# Baixar do Kaggle

Adicionei essa sessão aqui caso você queira baixar direto do Kaggle, mas para isso precisa criar um token etc. Se quiser explorar essa opção, veja esse [tutorial](https://www.freecodecamp.org/news/how-to-download-kaggle-dataset-to-google-colab/).

Se não quiser fazer assim, pode ignorar e ir direto para a forma que fez usando o Drive

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kaggle
kaggle.api.authenticate()
kaggle.api.dataset_download_files('muratkokludataset/date-fruit-datasets', path='/content/')

In [ ]:
! unzip date-fruit-datasets.zip

In [ ]:
file_path = '/content/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx'

# Dataset do Drive

In [ ]:
# conectando com o google drive e adicionando o path para o dataset
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Dataset/Date_Fruit_Datasets.xlsx'

# Montando DF

In [ ]:
# Leitura do dataset
df = pd.read_excel(file_path)
df.head()

In [ ]:
def encode (df):
    for col in df.columns:
        if df[col].dtype == 'object' or df[col].dtype == 'category':
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
    return df
df = encode(df)

In [ ]:
df.head()

In [ ]:
# Separando X e Y
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Importante
Eu adicionei aqui um StandardScaler, o que isso faz é padronizar os dados, ou seja, fazer a normalização deles.

Você deve ter percebido que os valores das colunas variam bastante, e como vimos nas aulas anteriores, temos que normalizar os dados para que o modelo consiga fazer o treinamento de maneira adequada. A normalização pode ser feita de muitas formas, como demonstrei em aula, e usando o StardardScaler é uma delas, similar ao MinMax. Leia essa referência que encontrei para relembrar [link](https://medium.com/@mhvasconcelos/distribui%C3%A7%C3%A3o-normal-e-a-biblioteca-standard-scaler-em-python-f21c52070c6b).



---



**Recomendo que tente fazer o treinamento com e sem o StandardScaler e veja a diferença que dá no treinamento. Faça isso e escreva um parágrafo no final do notebook explicando o que aconteceu.**



---



Para não usar o StandardScaler, basta não executar o trecho de código abaixo

In [ ]:
X = StandardScaler().fit_transform(X)
X[:5]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# pegando o shape da entrada para saber como montar a primeira camada da rede
# como temos 34 colunas, isso vai refletir na primeira camada
X_train_tensor.shape

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

## Criação da rede

Não recomendo usar rede convolucional pois não estudamos isso a fundo, por conta disso, troquei por uma rede neural tradicional densa.

Você deve perceber que a primeira camada, **fc1**, tem como entrada 34 colunas, que são o número de colunas que tem o X do dataset.

Adicionalmente, a última camada, **fc4**, tem como saída 7 possibilidades, que são o número de possíveis classes de saída.

Basicamente são esses os parâmetros que precisam bater com o dataset, os outros valores da rede podem ser mudados.

``` python
self.fc1 = nn.Linear(34, 256)
...
self.fc4 = nn.Linear(64, 7)
```

A rede que coloquei é exemplo, se quiser, pode deixar ela mais ou menos complexa adicionando ou removendo camadas e neurõnios.



In [ ]:
class Net(nn.Module):
    '''
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    '''
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(34, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 7)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net()

# Predict

In [ ]:
model(X_train_tensor)

## Treinamento

### Criando o objeto de treinamento

In [ ]:
def train(log_interval, dry_run, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Avaliação

In [ ]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(1111)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 64}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

train_loader = torch.utils.data.DataLoader(train_dataset,**train_kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1)

epochs = 14
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

for epoch in range(1, epochs + 1):
    train(10, False, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "date_fruit_cnn.pt")